# Исследование объявлений о продаже квартир

В вашем распоряжении данные сервиса Яндекc Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктах за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. Ваша задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность. 

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма. 

## Изучение данных из файла

Извлечём данные, посмотрим начало таблицы и общую информацию.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime


In [2]:
data = pd.read_csv('real_estate_data.csv', sep='\t')
pd.set_option('display.max_columns', 50) 

data.head(10)

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.00,2019-03-07T00:00:00,3,2.70,16.0,51.00,8,NaN,False,False,25.00,NaN,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN
1,7,3350000.0,40.40,2018-12-04T00:00:00,1,NaN,11.0,18.60,1,NaN,False,False,11.00,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
2,10,5196000.0,56.00,2015-08-20T00:00:00,2,NaN,5.0,34.30,4,NaN,False,False,8.30,0.0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0
3,0,64900000.0,159.00,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,NaN,False,False,NaN,0.0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0
4,2,10000000.0,100.00,2018-06-19T00:00:00,2,3.03,14.0,32.00,13,NaN,False,False,41.00,NaN,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0
5,10,2890000.0,30.40,2018-09-10T00:00:00,1,NaN,12.0,14.40,5,NaN,False,False,9.10,NaN,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55.0
6,6,3700000.0,37.30,2017-11-02T00:00:00,1,NaN,26.0,10.60,6,NaN,False,False,14.40,1.0,посёлок Парголово,52996.0,19143.0,0.0,NaN,0.0,NaN,155.0
7,5,7915000.0,71.60,2019-04-18T00:00:00,2,NaN,24.0,NaN,22,NaN,False,False,18.90,2.0,Санкт-Петербург,23982.0,11634.0,0.0,NaN,0.0,NaN,NaN
8,20,2900000.0,33.16,2018-05-23T00:00:00,1,NaN,27.0,15.43,26,NaN,False,False,8.81,NaN,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189.0
9,18,5400000.0,61.00,2017-02-26T00:00:00,3,2.50,9.0,43.60,7,NaN,False,False,6.50,2.0,Санкт-Петербург,50898.0,15008.0,0.0,NaN,0.0,NaN,289.0


In [3]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
total_images,23699.0,9.858475e+00,5.682529e+00,0.0,6.00,9.00,14.0,50.0
last_price,23699.0,6.541549e+06,1.088701e+07,12190.0,3400000.00,4650000.00,6800000.0,763000000.0
total_area,23699.0,6.034865e+01,3.565408e+01,12.0,40.00,52.00,69.9,900.0
rooms,23699.0,2.070636e+00,1.078405e+00,0.0,1.00,2.00,3.0,19.0
ceiling_height,14504.0,2.771499e+00,1.261056e+00,1.0,2.52,2.65,2.8,100.0
floors_total,23613.0,1.067382e+01,6.597173e+00,1.0,5.00,9.00,16.0,60.0
living_area,21796.0,3.445785e+01,2.203045e+01,2.0,18.60,30.00,42.3,409.7
floor,23699.0,5.892358e+00,4.885249e+00,1.0,2.00,4.00,8.0,33.0
kitchen_area,21421.0,1.056981e+01,5.905438e+00,1.3,7.00,9.10,12.0,112.0
balcony,12180.0,1.150082e+00,1.071300e+00,0.0,0.00,1.00,2.0,5.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23699 entries, 0 to 23698
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_images          23699 non-null  int64  
 1   last_price            23699 non-null  float64
 2   total_area            23699 non-null  float64
 3   first_day_exposition  23699 non-null  object 
 4   rooms                 23699 non-null  int64  
 5   ceiling_height        14504 non-null  float64
 6   floors_total          23613 non-null  float64
 7   living_area           21796 non-null  float64
 8   floor                 23699 non-null  int64  
 9   is_apartment          2775 non-null   object 
 10  studio                23699 non-null  bool   
 11  open_plan             23699 non-null  bool   
 12  kitchen_area          21421 non-null  float64
 13  balcony               12180 non-null  float64
 14  locality_name         23650 non-null  object 
 15  airports_nearest   

In [5]:
data.duplicated().sum()

0

### Вывод

- много пустых значений
- нет одинообразия в названиях столбцов
- некорректные типы данных

Дубликатов нет. Но, возможно, одна и та же квартира выставлялась на продажу несколько раз в разное время, а значит и цена тоже могла меняться.

## Предобработка данных

Начнём с названий столбцов

In [6]:
data.columns.tolist()

['total_images',
 'last_price',
 'total_area',
 'first_day_exposition',
 'rooms',
 'ceiling_height',
 'floors_total',
 'living_area',
 'floor',
 'is_apartment',
 'studio',
 'open_plan',
 'kitchen_area',
 'balcony',
 'locality_name',
 'airports_nearest',
 'cityCenters_nearest',
 'parks_around3000',
 'parks_nearest',
 'ponds_around3000',
 'ponds_nearest',
 'days_exposition']

In [7]:
data=data.rename(columns = {'total_area':'total_area_m2',
                       'studio':'is_studio',
                       'open_plan':'is_open_plan',
                       'locality_name':'town_name',
                       'cityCenters_nearest':'city_center_distance',
                       'parks_around3000':'parks_around_3km',
                       'ponds_around3000':'ponds_around_3km'
                      })

Сделано

In [8]:
data.isnull().sum().sort_values()

total_images                0
is_open_plan                0
floor                       0
rooms                       0
is_studio                   0
total_area_m2               0
last_price                  0
first_day_exposition        0
town_name                  49
floors_total               86
living_area              1903
kitchen_area             2278
days_exposition          3181
ponds_around_3km         5518
parks_around_3km         5518
city_center_distance     5519
airports_nearest         5542
ceiling_height           9195
balcony                 11519
ponds_nearest           14589
parks_nearest           15620
is_apartment            20924
dtype: int64

Не все пропуски выглядят критично для анализа. Например, люди могут просто не знать, какое расстояние до ближайшего парка или до аэропорта. Некоторые параметры люди могут просто полениться заполнять при размещении объявления.

Поменяем столбцы местами, чтобы было удобнее изучать таблицу в дальнейшем.

In [9]:
data = data[[
 'town_name',  
 'total_area_m2',
 'living_area',
 'kitchen_area',
 'ceiling_height',
 'rooms',
 'balcony',
 'floor',
 'floors_total',
 'is_apartment',
 'is_studio',
 'is_open_plan',  
 'total_images',
 'last_price',
 'first_day_exposition',
 'days_exposition',
 'city_center_distance',
 'airports_nearest',
 'parks_nearest',
 'parks_around_3km',
 'ponds_nearest',
 'ponds_around_3km'
]]

In [10]:
data.head()

,town_name,total_area_m2,living_area,kitchen_area,ceiling_height,rooms,balcony,floor,floors_total,is_apartment,is_studio,is_open_plan,total_images,last_price,first_day_exposition,days_exposition,city_center_distance,airports_nearest,parks_nearest,parks_around_3km,ponds_nearest,ponds_around_3km
0,Санкт-Петербург,108.0,51.0,25.0,2.70,3,NaN,8,16.0,NaN,False,False,20,13000000.0,2019-03-07T00:00:00,NaN,16028.0,18863.0,482.0,1.0,755.0,2.0
1,посёлок Шушары,40.4,18.6,11.0,NaN,1,2.0,1,11.0,NaN,False,False,7,3350000.0,2018-12-04T00:00:00,81.0,18603.0,12817.0,NaN,0.0,NaN,0.0
2,Санкт-Петербург,56.0,34.3,8.3,NaN,2,0.0,4,5.0,NaN,False,False,10,5196000.0,2015-08-20T00:00:00,558.0,13933.0,21741.0,90.0,1.0,574.0,2.0
3,Санкт-Петербург,159.0,NaN,NaN,NaN,3,0.0,9,14.0,NaN,False,False,0,64900000.0,2015-07-24T00:00:00,424.0,6800.0,28098.0,84.0,2.0,234.0,3.0
4,Санкт-Петербург,100.0,32.0,41.0,3.03,2,NaN,13,14.0,NaN,False,False,2,10000000.0,2018-06-19T00:00:00,121.0,8098.0,31856.0,112.0,2.0,48.0,1.0


Проверим столбец с названиями населённых пунктов.

In [11]:
data['town_name'].sort_values().unique().tolist()


['Бокситогорск',
 'Волосово',
 'Волхов',
 'Всеволожск',
 'Выборг',
 'Высоцк',
 'Гатчина',
 'Зеленогорск',
 'Ивангород',
 'Каменногорск',
 'Кингисепп',
 'Кириши',
 'Кировск',
 'Колпино',
 'Коммунар',
 'Красное Село',
 'Кронштадт',
 'Кудрово',
 'Лодейное Поле',
 'Ломоносов',
 'Луга',
 'Любань',
 'Мурино',
 'Никольское',
 'Новая Ладога',
 'Отрадное',
 'Павловск',
 'Петергоф',
 'Пикалёво',
 'Подпорожье',
 'Приморск',
 'Приозерск',
 'Пушкин',
 'Санкт-Петербург',
 'Светогорск',
 'Сертолово',
 'Сестрорецк',
 'Сланцы',
 'Сосновый Бор',
 'Сясьстрой',
 'Тихвин',
 'Тосно',
 'Шлиссельбург',
 'городской поселок Большая Ижора',
 'городской поселок Янино-1',
 'городской посёлок Будогощь',
 'городской посёлок Виллози',
 'городской посёлок Лесогорский',
 'городской посёлок Мга',
 'городской посёлок Назия',
 'городской посёлок Новоселье',
 'городской посёлок Павлово',
 'городской посёлок Рощино',
 'городской посёлок Свирьстрой',
 'городской посёлок Советский',
 'городской посёлок Фёдоровское',
 'городск

In [12]:
len(data['town_name'].unique())

365

Ад и погибель...
Без 100 грамм не разобраться. Попробуем уменьшить количество названий заменив "ё" на "е".

In [13]:
data['town_name'] = data['town_name'].str.replace('ё', 'е', regex=True)
len(data['town_name'].unique())

331

In [14]:
data['town_name'].isna().sum()

49

Стало меньше. Мелочь а приятно. Судя по всему недвижимость пользуется спром не только в Питере и крупных городах Ленинградской области, но и в маленьких деревнях.

При этом объявления без указания населенного пункта бесполезны для потенциальных покупателей, поэтому такие записи можно сразу удалить без зазрения совести.

In [15]:
data = data.dropna(subset=['town_name'])
data['town_name'].isna().sum()

0

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23650 entries, 0 to 23698
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   town_name             23650 non-null  object 
 1   total_area_m2         23650 non-null  float64
 2   living_area           21752 non-null  float64
 3   kitchen_area          21381 non-null  float64
 4   ceiling_height        14490 non-null  float64
 5   rooms                 23650 non-null  int64  
 6   balcony               12137 non-null  float64
 7   floor                 23650 non-null  int64  
 8   floors_total          23565 non-null  float64
 9   is_apartment          2760 non-null   object 
 10  is_studio             23650 non-null  bool   
 11  is_open_plan          23650 non-null  bool   
 12  total_images          23650 non-null  int64  
 13  last_price            23650 non-null  float64
 14  first_day_exposition  23650 non-null  object 
 15  days_exposition    

In [17]:
data['is_apartment'].unique()

array([nan, False, True], dtype=object)

Параметр "is_apartment" явно отвечает на коварный вопрос "А не является ли апартаментами недвижимость в объявлении?"
А значит значение NaN как бы намекает, что нет. Значит пустые значения очень даже можно заменить на значение False. 

In [18]:
data['is_apartment'] = data['is_apartment'].fillna(0)

Тяга к прекрасному и любовь к цифрам заставляет меня привести этот столбец к набору цифр 0 и 1.

In [19]:
data['is_apartment'] = data['is_apartment'].map({True: 1, False: 0})
data.head()

,town_name,total_area_m2,living_area,kitchen_area,ceiling_height,rooms,balcony,floor,floors_total,is_apartment,is_studio,is_open_plan,total_images,last_price,first_day_exposition,days_exposition,city_center_distance,airports_nearest,parks_nearest,parks_around_3km,ponds_nearest,ponds_around_3km
0,Санкт-Петербург,108.0,51.0,25.0,2.70,3,NaN,8,16.0,0,False,False,20,13000000.0,2019-03-07T00:00:00,NaN,16028.0,18863.0,482.0,1.0,755.0,2.0
1,поселок Шушары,40.4,18.6,11.0,NaN,1,2.0,1,11.0,0,False,False,7,3350000.0,2018-12-04T00:00:00,81.0,18603.0,12817.0,NaN,0.0,NaN,0.0
2,Санкт-Петербург,56.0,34.3,8.3,NaN,2,0.0,4,5.0,0,False,False,10,5196000.0,2015-08-20T00:00:00,558.0,13933.0,21741.0,90.0,1.0,574.0,2.0
3,Санкт-Петербург,159.0,NaN,NaN,NaN,3,0.0,9,14.0,0,False,False,0,64900000.0,2015-07-24T00:00:00,424.0,6800.0,28098.0,84.0,2.0,234.0,3.0
4,Санкт-Петербург,100.0,32.0,41.0,3.03,2,NaN,13,14.0,0,False,False,2,10000000.0,2018-06-19T00:00:00,121.0,8098.0,31856.0,112.0,2.0,48.0,1.0


Красиво?

Красиво!

Но не будем останавливаться на достигнутом и провернём то же самое для столбцов "is_studio" и "is_open_plan".

In [20]:
data['is_studio'] = data['is_studio'].map({True: 1, False: 0})
data['is_open_plan'] = data['is_open_plan'].map({True: 1, False: 0})
data.head().T

,0,1,2,3,4
town_name,Санкт-Петербург,поселок Шушары,Санкт-Петербург,Санкт-Петербург,Санкт-Петербург
total_area_m2,108.0,40.4,56.0,159.0,100.0
living_area,51.0,18.6,34.3,NaN,32.0
kitchen_area,25.0,11.0,8.3,NaN,41.0
ceiling_height,2.7,NaN,NaN,NaN,3.03
rooms,3,1,2,3,2
balcony,NaN,2.0,0.0,0.0,NaN
floor,8,1,4,9,13
floors_total,16.0,11.0,5.0,14.0,14.0
is_apartment,0,0,0,0,0


Время проверки балконов!

In [21]:
data['balcony'].unique()

array([nan,  2.,  0.,  1.,  5.,  4.,  3.])

Правдоподобно, нет отрицательных и слишком больших значений. Заменить nan на 0 и можно продолжать дальше.

In [22]:
data['balcony'] = data['balcony'].fillna(0)
data['balcony'].value_counts().sort_values()

3.0       81
4.0      182
5.0      304
2.0     3658
1.0     4187
0.0    15238
Name: balcony, dtype: int64

Балконы красавцы! Чего не скажешь о столбце с датой публицкации объявления. Сейчас там текстовое значение. Приведём его во временной формат

In [23]:
data['first_day_exposition'] = pd.to_datetime(data['first_day_exposition'], format="%Y-%m-%dT%H:%M:%S")
data.head()

,town_name,total_area_m2,living_area,kitchen_area,ceiling_height,rooms,balcony,floor,floors_total,is_apartment,is_studio,is_open_plan,total_images,last_price,first_day_exposition,days_exposition,city_center_distance,airports_nearest,parks_nearest,parks_around_3km,ponds_nearest,ponds_around_3km
0,Санкт-Петербург,108.0,51.0,25.0,2.70,3,0.0,8,16.0,0,0,0,20,13000000.0,2019-03-07,NaN,16028.0,18863.0,482.0,1.0,755.0,2.0
1,поселок Шушары,40.4,18.6,11.0,NaN,1,2.0,1,11.0,0,0,0,7,3350000.0,2018-12-04,81.0,18603.0,12817.0,NaN,0.0,NaN,0.0
2,Санкт-Петербург,56.0,34.3,8.3,NaN,2,0.0,4,5.0,0,0,0,10,5196000.0,2015-08-20,558.0,13933.0,21741.0,90.0,1.0,574.0,2.0
3,Санкт-Петербург,159.0,NaN,NaN,NaN,3,0.0,9,14.0,0,0,0,0,64900000.0,2015-07-24,424.0,6800.0,28098.0,84.0,2.0,234.0,3.0
4,Санкт-Петербург,100.0,32.0,41.0,3.03,2,0.0,13,14.0,0,0,0,2,10000000.0,2018-06-19,121.0,8098.0,31856.0,112.0,2.0,48.0,1.0


Переведём расстояния в километры

In [25]:
data['city_center_distance'] = data.apply(lambda x: data['city_center_distance'] / 1000)
data['airports_nearest'] = data.apply(lambda x: data['airports_nearest'] / 1000)
data['parks_nearest'] = data.apply(lambda x: data['parks_nearest'] / 1000)
data['ponds_nearest'] = data.apply(lambda x: data['ponds_nearest'] / 1000)
data.head()



ValueError: Columns must be same length as key

In [ ]:
data['ceiling_height'].isnull().sum()

Много нулевых значений для высоты потолков в квартирах. Что-то здесь не так. Скорее всего люди просто не указывают эти данные при заполнении параметров объявления. Если высота потолков большая, то её скорее всего укажут, так как это может быть большим плюсом при выборе квартиры. Значит почти наверняка неуказанные данные где-то в районе среднего значения. Для данного параметра среднее значение равно 2,77, а медианное 2,65. Но на среднее очень влияют выбросы (а-ля 100 метров в максимуме). Поэтому возьмём медианное 2,65

In [ ]:
height_median = data['ceiling_height'].median()

data['ceiling_height'].fillna(value = height_median, inplace=True)
data['ceiling_height'].isnull().sum()

Аналогичную операцию проведём и для жилой площади и площади кухни.

In [ ]:
living_area_median = data['living_area'].median()

data['living_area'].fillna(value = living_area_median, inplace=True)
data['living_area'].isnull().sum()

In [ ]:
kitchen_area_median = data['living_area'].median()

data['kitchen_area'].fillna(value = kitchen_area_median, inplace=True)
data['kitchen_area'].isnull().sum()

**Вывод**

- Переименованы и поменяны местами для удобства просмотра некоторые столбцы
- Заполнены пустые значения, где это возможно.
- Скорректированы названия населённых пунктов (убрана буква Ё)

## Расчёты и добавление результатов в таблицу

Посчитаем стоимость квадратного метра, исходя из площади и общей стоимости квартиры.

In [ ]:
data['price_per_meter'] = data['last_price'] / data['total_area_m2']
data['price_per_meter'] = data['price_per_meter'].round(decimals=2)
data.head()

Добавим день недели, месяц и год публикации объявления.

In [ ]:
data['day_of_week_exposition'] = data['first_day_exposition'].dt.day_name()
data['month_exposition'] = data['first_day_exposition'].dt.month_name()
data['year_exposition'] = data['first_day_exposition'].dt.year
data.head()

Добавим данные по этажам (первый/последний/другой). Напишем для этого отдельную функцию.

In [ ]:
def floor_status(row):
    if row['floor'] == 1:
        return 'первый'
    elif row['floor'] == row['floors_total']:
        return 'последний'
    else:
        return 'другой'

In [ ]:
data['floor_status'] = data.apply(floor_status, axis=1)
data.head()

Вычислим соотношение жилой и общей площади, а также отношение площади кухни к общей

In [ ]:
data['total_living_area_ratio'] = data['living_area'] / data['total_area_m2']
data['total_kitchen_area_ratio'] = data['kitchen_area'] / data['total_area_m2']
data.head()

**Вывод**

Добавлены новые столбцы:
- price_per_meter
- day_of_week_exposition
- month_exposition
- year_exposition
- total_living_area_ratio
- total_kitchen_area_ratio

## Исследовательский анализ данных

Ряд данных в процессе исследования показались необычными. Например:
- площадь квартиры 900 кв.м.
- высота потолка 1 и 100 м.
- количество этажей в доме - 60.

Напавшее вдохновение требует автоматизировать процесс поиска выбросов и пересчёта

### Анализ площади, цены, числа комнат, высоты потолков.

In [ ]:
data.plot(kind = 'hist',
          y = 'total_area_m2',
          bins = 150, 
          label = 'Общая площадь квартир',
          range = (0, 250),
          grid=True,
          legend=True)

Большая часть квартир по площади в пределах 150 кв. м. Самое большое число квартир с площадью до 50 кв. м. Очевидно, что это самые ходовы квартиры, которых и строится, и продаётся больше всего.

Локальные пики на графике можно объяснить тем, что площади строившихся квартир как правило имеют жесткие значения, привязанные к серии дома и не меняются годами. Квартиры в домах популярных проектных серий встречаются намного чаще.

In [ ]:
data['last_price'] = data.apply(lambda x: data['last_price'] / 1000000)


Перевели стоимость квартиры в миллионы

In [ ]:
data.plot(kind = 'hist',
          y= 'last_price',
          bins = 100, 
          label = 'Стоимость квартир',
          range = (0,800),
          grid=True,
          legend=True)

Из-за некоторого количества очень дорогих квартир (самая дорогая стоит 763 млн) график выглядит таким образом. Отсечём очень дорогие квартиры (стоимостью больше 100 млн)

In [ ]:
(data
     .query('last_price < 100')
     .plot(kind = 'hist',
          y= 'last_price',
          bins = 100, 
          label = 'Стоимость квартир',
          range = (0,100),
          grid=True,
          legend=True)
)

Любо-дорого посмотреть! (причём не столько любо, сколько дорого)

Для большей части квартир их стоимость лежит в пределах 10 млн рублей.

In [ ]:
data.plot(kind = 'hist',
          y= 'rooms',
          bins = 10, 
          label = 'Число комнат',
          range = (0,19),
          grid=True,
          legend=True)

Больше всего 3-комнатных и 2-комнатных квартир. Поле них по частоте идут однокомнатные.

In [ ]:
(data
     .query('ceiling_height < 10 & ceiling_height > 1.8')
     .plot(kind = 'hist',
          y= 'ceiling_height',
          bins = 100, 
          label = 'Высота потолков',
          range = (1,10),
          grid=True,
          legend=True)
)

Большая часть значений высоты потолка сконцентрирована в районе 2,5-3 метров. Экстремально маленькие и большие значения мы отсекли запросом перед построением графика.

In [ ]:
data.plot(kind = 'hist',
          y= 'days_exposition',
          bins = 100, 
          label = 'Срок продажи',
          #range = (0,20),
          grid=True,
          legend=True)

### Время продажи квартиры

In [ ]:
data['days_exposition'].describe()

Большинство квартир продаётся в первые полгода с момента выставления на продажу. Нахождение квартиры в базе недвижимости более 500 дней можно считать аномалией.
Среднее значение срока продажи 180 дней. Медианное - 95 дней.

In [ ]:
data.query('days_exposition > 500')['days_exposition'].count()

Это всего 7% от общего числа записей в базе.

### Чистим от выбросов, которые мешают анализу

Соберём новый датасет, убрав следующие выбросы:
- цена выше 100 млн рублей.
- высота потолков ниже 2,2 м (чтобы даже у высоких людей был шанс не дотянуться рукой до потолка)
- высока потолков выше 5,5 м (могут ведь и двухуровненые квартиры)
- срок продажи квартиры более 500 дней (полтора года для продажи более чем достаточно)
- число комнат в квартире больше 10 (такие значения чаще появляются из-за опечаток, но очень влияют на средние и медианные значения)
- жилая площадь меньше 10 кв.м. (скорее всего тоже опечатки)

In [ ]:
good_data = data.query('last_price <= 100 \
                       and 2.2 <= ceiling_height <= 5.5 \
                       and days_exposition <= 500 \
                       and rooms <= 10 \
                       and living_area >= 10')
good_data.info()

In [ ]:
good_data.describe().T

Данных стало меньше, но теперь они более точные, так как очищены от выбросов. Можно перейти к выявлению зависимостей между параметрами.

### Посмотрим, зависит ли цена от площади, числа комнат, удалённости от центра

In [ ]:
corr = good_data[['last_price',\
                  'total_area_m2',\
                  'rooms',\
                  'city_center_distance']].corr()
corr.style.background_gradient(cmap='coolwarm')

Для проверки себя посмотрим на диаграмму рассеяния для оценки зависимости стоимости цены от площади, количества комнат и расстояния до центра.

In [ ]:
good_data.plot(x='last_price', y='total_area_m2', kind='scatter', grid=True, alpha = 0.3)

In [ ]:
good_data.plot(x='last_price', y='rooms', kind='scatter', grid=True, alpha = 0.3)

In [ ]:
good_data.plot(x='last_price', y='city_center_distance', kind='scatter', grid=True, alpha = 0.3)

Получается логичная картина - конечная цена квартира в максимальной степени напрямую зависит от общей площади. От числа комнат тоже, но не так сильно. При этом зависимость от расстояния от центра города обратная, но самая слабая по всем трём параметрам.

Посмотрим на зависимость цен от этажа (первый/последний/другой), дней недели, месяцев и годов. Но теперь воспользуемся другой формой диаграмм.

In [ ]:
plot = good_data.boxplot('last_price', by='floor_status',figsize=(10,10)).set_ylim(0,20)

Тоже логичный вывод. Меньше всего средняя стоимость у квартир на первом этаже. Спрос на них и правда ниже в реальности.

In [ ]:
plot = good_data.boxplot('last_price', by='day_of_week_exposition',figsize=(10,10)).set_ylim(0,20)

In [ ]:
plot = good_data.boxplot('last_price', by='month_exposition',figsize=(10,10)).set_ylim(0,20)

In [ ]:
plot = good_data.boxplot('last_price', by='year_exposition',figsize=(10,10)).set_ylim(0,20)

А вот зависимость от дня недели и месяца не прослеживается. При этом можно заметить небольшие колебания средних и медианных цен по разным годам. Но на это больше влияют макроэкономиеческие факторы в стране и мире в целом. В нашем анализе они не рассматриваются.

### Посмотрим на населенные пункты с самым дорогим жильём

Выберем 10 населённых пунктов с наибольшим числом объявлений. Посчитаем среднюю цену квадратного метра в этих населённых пунктах. Выделим населённые пункты с самой высокой и низкой стоимостью жилья.

In [ ]:
(good_data['town_name'].value_counts()).head(10)

Для определения самой дорогой недвижимости логично сравнивать не абсолютную стоимость квартир, а среднюю стоимость квадратного метра по данным населенным пунктам.

Для этого введём новый датафрейм с набором данных по 10 городам.

In [ ]:
town_data = (good_data
     .groupby('town_name')\
     .agg({'price_per_meter':'mean'})
     .sort_values('price_per_meter', ascending=False)
     .head(10)

     
)
 

town_data['town_name'] = town_data.index
town_data

In [ ]:
town_data.plot(x='town_name', y='price_per_meter', kind='bar', grid=True)

Лисий Нос и Зеленогорск это дорогие районы по отзывам знакомых из Питера. Так что стоимость квадратного метра там вполне может быть сходна со стоимость жилья в Питере.

### Теперь разберёмся с центром Питера.

Вернёмся к исходным неочищенным данным. Посмотрим, в скольких объявлениях вообще указано расстояние до центра.

In [ ]:
data['city_center_distance'].isnull().sum()

Много нулевых значений (чуть меньше четверти исходного набора данных). Чем заполнять непонятно. Квартира может быть как в центре, так и на окраине. А значит медиана или среднее значение испортит нам данные для анализа. К тому же люди, размещая объявления, могут просто не знать расстояние до центра.
Поэтому для решения этой задачи соберём новый набор данных, удалив записи с пустым значением расстояния до центра.

In [ ]:
city_center_data = data.query('town_name == "Санкт-Петербург"')
city_center_data.dropna(subset=['city_center_distance'])
city_center_data.info()

Добавим столбец с целочисленным расстоянием до центра

In [ ]:
city_center_data['city_center_distance_round'] = city_center_data['city_center_distance'].round(0)
city_center_data.head(10)

In [ ]:
city_center_data.plot(x='city_center_distance_round', y='price_per_meter', kind='scatter', alpha=0.2) 

Визуально видно, что заметный скачок цены за квадратный метр возникает примерно в радиусе 7 км от центра.
Выделим эту группу объявления в отдельный датафрейм.

Есть подозрение, что именно в пределах 7 км уже не так важно точное расстояние до центра и зависимость стоимости квартиры от расстояния станет меньше.

In [ ]:
city_center_data_7 = city_center_data.query('city_center_distance < 7')
city_center_data_7.info()

Соберём в таблицу данные для построения графика зависимости стоимости квадратного метра от расстояния до центра в пределах 7 км.

In [ ]:
city_center_data_7_graphic = city_center_data_7.pivot_table(index='city_center_distance_round',\
                                                            values='price_per_meter', \
                                                           aggfunc='mean')
city_center_data_7_graphic

In [ ]:
city_center_data_7_graphic.plot(y='price_per_meter',\
                                label='Средняя стоимость квадратного метра')

Очевидно самое дорогое жильё в самом центре города. Необычным выглядит провал на 3 км от центра. Возможно, это объясняется какой-то особенностью расположения районом Санкт_Петербурга. Лучше об этом могут рассказать риэлторы, работающие на этом рынке.

Посмотрим следующие параметры: площадь, цена, число комнат, высота потолков.

In [ ]:
corr = city_center_data_7[['last_price',\
                           'total_area_m2',\
                           'rooms',\
                           'city_center_distance_round',\
                           'days_exposition',\
                           'ceiling_height']].corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
plot = city_center_data_7.boxplot('last_price', by='floor_status',figsize=(10,10)).set_ylim(0,100)

In [ ]:
plot = city_center_data_7.boxplot('last_price', by='day_of_week_exposition',figsize=(10,10)).set_ylim(0,100)

In [ ]:
plot = city_center_data_7.boxplot('last_price', by='month_exposition',figsize=(10,10)).set_ylim(0,100)

In [ ]:
plot = city_center_data_7.boxplot('last_price', by='year_exposition',figsize=(10,10)).set_ylim(0,120)

**Вывод**

В центральном районе Питера цена прямопропорционально зависит от общей площади. Также есть корреляция (хоть и в меньшей степени от числа комнат), что не менее логично, так как большое число комнат почти наверняка означает большую площадь квартира.

При это от высоты потолков, расстояния до центра (не забываем, что мы и так уже в центре) и срока размещения объявления цена квартир практически независит.

Отличия центральной выборки от общей всё-таки заметны. Здесь влияние на цену квадратного метра всех факторов заметно уменьшается. Явно из-за того, что весь выбранный район уже является "элитным", а значит выбранный порог вхождения квартир в центр города выбран корректно.

Зависимость цены квартиры от этажа, дня недели и месяца такая же как и для всего города. А вот график для годов заметно отличается. В 2014 году цены на квартиры в центре отличались от средних по городу гораздо больше. В последующие года средние цены в центре города заметно снизились. Скорее всего это связано с какими-то макроэкономическими факторами, повлиявшими на рынок жилья в Санкт-Петербурге.

## Общий вывод

Общие данные базы, взятой для анализа:
- Число строк: **23699**
- Число колонок: 22
- Объём занимаемой памяти: 3,7 Мб

Главной задачей анализа было выявление параметров, максимально влияющие на итоговую цену квартиру. Поэтому главным параметром в таблице является итоговая цена, а остальные параметры проверялись на корреляцию с итоговой ценой.


Задача состояла в проверке данных на предмет качества выгрузки из системы. В ходе первичного анализа были выявлены и решены следующие проблемы:

- пропущенные значения (заменены средним, медианным или нулевым значением в зависимости от каждого конкретного параметра)
- исправлены имена столбцов и часть значений в столбце с названиями населённых пунктов (замена е/ё)
- расстояния приведены в удобную форму значения расстояний (метры переведены в километры)
- добавлены цена за квадратный метр, значение этажа, день/месяц/год размещения объявления, соотношение жилой площади и площади кухни к общей площади.

Кроме ошибочных и недостающих данных были выявлены также выбросы в данных, которые не могут существовать в реальности (например высота потолков в 1 или 100 метров). Чтобы эти данные не влияли на средние и медианные значения в ходе анализа было решено удалиь эти данные как некорректные. В скорректированном датафрейме осталось **18642** строк, т.е. примерно **78%** от исходного. Это может говорить о недостаточном качестве исходной базы. Предварительная рекомендация - **добавить в систему проверку заведомо ложных значений при добавлении объявления**, посколько характер ошибок позволяет предположить, что они сделаны по ошибке пользователем, а значит их можно обрабатывать на этапе ввода данных.


Проведён анализ предложенной базы недвижимости для оценки зависимости стоимости квартиры от следующих параметров:
- общая площадь
- число комнат
- расстояние до центра города
- высота потолков
- срок размещения объявления

**Выявлено**
Самое дорогое жилье (максимальная средняя стоимость квадратного метра) в трёх населённых пунктах: поселок Лисий Нос,
Зеленогорск и Санкт-Петербург. Для них средняя стоимость квадратного метра ***более 110 тысяч рублей***. Масимальное влияние на цену квартиры очевидно оказывают общая площадь, число комнат и удалённость из центра. Причём условной "границей" центра можно считать радиус 7 км от географического центра города. Именно внутри радиуса в 7 км влияние расстояния на цену квартиры падает. А при выходе за этот радиус наблюдается обратная зависимость - чем дальше от центра, тем меньше цена.


